In [1]:
import pickle 
import nltk
import pertubations_helper as ph

with open('ne_collection.obj', 'rb') as f:
    ne_collection = pickle.load(f)

with open('pos_collection.obj', 'rb') as f:
    pos_collection = pickle.load(f)

In [2]:
from datasets import load_dataset

squad = load_dataset('squad_v2')

/Users/v/miniconda/envs/mthesis/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset squad_v2 (/Users/v/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 198.08it/s]


In [3]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/bert-small-finetuned-squadv2",
    tokenizer="mrm8488/bert-small-finetuned-squadv2"
)

qa_pipeline({
    'context': "Manuel Romero has been working hardly in the repository hugginface/transformers lately",
    'question': "Who has been working hard for hugginface/transformers lately?"

})

{'score': 0.9939450621604919, 'start': 0, 'end': 13, 'answer': 'Manuel Romero'}

In [4]:
import spacy
from spacy import displacy

ner = spacy.load('en_core_web_sm')

In [5]:
import statistics

print( statistics.mean( [len(ne_collection[k]) for k in ne_collection]) )
print( statistics.mean( [len(pos_collection[k]) for k in pos_collection]) )

5094.055555555556
5091.84


In [6]:
import numpy as np

n = 32

beam_size = 6

new_questions = []
for i, q_ctx in enumerate(squad['train']):
    question = q_ctx['question']
    context = q_ctx['context']
    
    score1 = qa_pipeline({'context': context, 'question': question})

    question_chunks = question.split()

    def is_attack_question( new_question ):
        score2 = qa_pipeline({'context': context, 'question': new_question})

        if ph.is_undersensitivity_attack( score1, score2 ):
            print( f'{question}({round(score1["score"],2)}) vs {new_question}({round(score2["score"],2)}) => {score1["answer"]}')
            return True
        
        return False

    new_question = ph.pos_pertubation( question, pos_collection, is_attack_question, beam_size=beam_size, n=n)
    new_questions.append(new_question)
    
    if len(new_questions) == 4:
        break
        

When did Beyonce start becoming popular?(0.38) vs When did Beyonce abstand sing elite(0.5) => late 1990s


KeyboardInterrupt: 

In [8]:
import numpy as np

n = 32

beam_size = 6

new_questions = []
for i, q_ctx in enumerate(squad['train']):
    question = q_ctx['question']
    context = q_ctx['context']
    
    score1 = qa_pipeline({'context': context, 'question': question})

    question_chunks = question.split()

    def is_attack_question( new_question ):
        score2 = qa_pipeline({'context': context, 'question': new_question})

        if ph.is_undersensitivity_attack( score1, score2 ):
            print( f'{question}({round(score1["score"],2)}) vs {new_question}({round(score2["score"],2)}) => {score1["answer"]}')
            return True
        
        return False

    while True:
        new_question = ph.ne_pertubation( question, ne_collection, ner, is_attack_question, beam_size=beam_size, n=n)
        if new_question != '':
            break
        
    new_questions.append(new_question)
    if len(new_questions) == 50:
        break

When did Beyonce start becoming popular?(0.38) vs When did Velas Latinas start becoming popular?(0.42) => late 1990s
What areas did Beyonce compete in when she was growing up?(0.65) vs What areas did Dunn compete in when she was growing up?(0.66) => singing and dancing
When did Beyonce leave Destiny's Child and become a solo singer?(0.84) vs When did the Arab League leave Warsh and become a solo singer?(0.87) => 2003
In what city and state did Beyonce  grow up? (0.67) vs In what city and state did Koninklijke Nederlandsche  grow up? (0.7) => Houston, Texas


KeyboardInterrupt: 